In [ ]:
import numpy as np
import os
from scipy.signal import lfilter
from scipy.linalg import toeplitz, solve_toeplitz
import scipy.io.wavfile as wav

lpc_order = 16

# Calculate LPC coefficients
def calculate_lpc(signal, order):
    # Autocorrelation method
    autocorr = np.correlate(signal, signal, mode='full')
    autocorr = autocorr[len(autocorr)//2:]
    
    # Solve the Yule-Walker equations
    R = toeplitz(autocorr[:order])
    r = autocorr[1:order+1]
    lpc_coeffs = solve_toeplitz((autocorr[:order], autocorr[:order]), r)
    
    return np.concatenate(([1], -lpc_coeffs))

# Directory containing the audio files
directory = 'no rain'

lpc_coeffs = []
# Process each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.wav'):
        filepath = os.path.join(directory, filename)
        
        # Read the audio file
        sr, signal = wav.read(filepath)
        
        # Extract a 10-second segment
        segment_samples = segment_duration * sr
        segment = signal[:segment_samples]
        
        # Calculate LPC coefficients for the segment
        lpc_coeffs.append(calculate_lpc(segment, lpc_order))     
